### Import Libraries

In [47]:
import pandas as pd
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.optimizers import Adadelta,SGD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,mean_squared_error,confusion_matrix
from sklearn.model_selection import GridSearchCV,train_test_split

### Load MNIST Dataset

In [20]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [21]:
# building the input vector from the 28x28 pixels
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [22]:
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

In [23]:
# print the final input shape ready for training
print("Train matrix shape", X_train.shape)
print("Test matrix shape", X_test.shape)

Train matrix shape (60000, 784)
Test matrix shape (10000, 784)


In [24]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


### Random Forest Classifier

In [25]:
# Random Forest Classifier
clf_rf = RandomForestClassifier(n_estimators=50)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
print("random forest accuracy: ",acc_rf)

random forest accuracy:  0.9672


### After Tweaking Parameters

In [26]:
rf = RandomForestRegressor(random_state = 42)
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [27]:
random_forest = RandomForestClassifier()
grid_search = GridSearchCV(estimator = random_forest, 
                           param_grid = {
                                            'max_depth': [60, 70],
                                            'n_estimators': [60, 70, 80]
                                        }, 
                          cv = 3, verbose = 1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  8.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [28]:
grid_search.best_params_

{'max_depth': 70, 'n_estimators': 80}

In [29]:
y_predict_gridsearchCV = grid_search.best_estimator_.predict(X_test)
y_train_predict_gridsearchCV = grid_search.best_estimator_.predict(X_train)
accuracy_rf = accuracy_score(y_predict_gridsearchCV,y_test)
loss_rf = mean_squared_error(y_predict_gridsearchCV,y_test)
print('Accuracy with GridSearchCV:',accuracy_rf)

Accuracy with GridSearchCV: 0.9682


### MLFNN

In [41]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_test = X_test.reshape(len(X_test),28*28)
X_train = X_train.reshape(len(X_train),28*28)

In [42]:
num_classes = 10
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)
y_valid = to_categorical(y_valid,num_classes)

In [35]:
num_classes = 10
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
print("Shape y-train before one-hot encoding: ", y_train.shape)
y_train = to_categorical(y_train,num_classes)
print("Shape y-train after one-hot encoding: ", y_train.shape)
y_test = to_categorical(y_test,num_classes)
Y_valid = to_categorical(y_valid,num_classes)

Shape y-train before one-hot encoding:  (38400,)
Shape y-train after one-hot encoding:  (38400, 10)


In [43]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_valid", X_valid.shape)
print("y_valid", y_valid.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (48000, 784)
y_train (48000, 10)
X_valid (12000, 784)
y_valid (12000, 10)
X_test (10000, 784)
y_test (10000, 10)


In [45]:
model = Sequential([Dense(32, input_shape=(784,)), Activation('sigmoid'), Dense(10), Activation('softmax')])

In [46]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 32)                25120     
_________________________________________________________________
activation_5 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(optimizer = SGD(), loss="categorical_crossentropy",metrics=['accuracy'])

In [49]:
history = model.fit(X_train, y_train, batch_size=128, epochs=50, validation_split=(X_valid, y_valid),verbose=1)

TypeError: '<' not supported between instances of 'float' and 'tuple'